# Constrained and guided generation with language models

This notebook explores the concepts of constrained and guided generation using large language models. Instead of letting the model freely generate text, we structure and restrict the generation process to produce more precise, rule-abiding, and goal-oriented outputs.

While LLMs like GPT are flexible and creative, that flexibility can sometimes be a drawback. Left to generate text freely, the model may:
- Drift off-topic.
- Produce verbose or inconsistent content.
- Fail to meet specific format or length requirements.
- Can be too creative when we need something structured.

This becomes especially problematic in real-world use cases where consistency, structure, and clarity are essential — such as in product descriptions, job postings, data extraction, and UI text generation.

This is where constrained generation shines — it helps us steer the model with a clear format, rules, and validation mechanisms. This approach not only improves the usability and accuracy of the generated text, but also makes LLMs better suited for integration into structured workflows and applications where format adherence is non-negotiable.

In [1]:
import os
import re

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import RegexParser

from dotenv import load_dotenv
load_dotenv()

# Set up the OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Initialize the language model
We instantiate a lightweight GPT model from OpenAI using LangChain.

In [2]:
# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18")

We will also define a helper function to neatly display outputs from the model:

In [3]:
# Function to display model outputs
def display_output(output):
    """Display the model's output in a formatted manner."""
    print("Model Output:")
    print("-" * 40)
    print(output)
    print("-" * 40)
    print()

## Setting up constraints for model outputs
The primary goal of constrained generation is to provide the model with a clear structure and specific guidelines that help produce outputs aligned with predefined requirements. This technique is particularly valuable when we need to generate text that adheres to certain formats or constraints, such as word count, tone, or specific content elements.

In this section, we will start by demonstrating how to create a constrained prompt that generates a product description with specific requirements.

In [4]:
# Define a prompt that constrains the model to follow a specific structure and tone
constrained_prompt = PromptTemplate(
    input_variables=["product", "target_audience", "tone", "word_limit"],
    template="""Create a product description for {product} targeted at {target_audience}.
    Use a {tone} tone and keep it under {word_limit} words.
    The description should include:
    1. A catchy headline
    2. Three key features
    3. A call to action

    Product Description:
    """
)

# Input variables that will be used to fill in the prompt template
input_variables = {
    "product": "smart water bottle",
    "target_audience": "health-conscious millennials",
    "tone": "casual and friendly",
    "word_limit": "75"
}

# Chain the prompt with the language model
chain = constrained_prompt | llm
# Generate the constrained output
output = chain.invoke(input_variables).content
display_output(output)

Model Output:
----------------------------------------
**Stay Hydrated, Stay Awesome!**  

Meet your new hydration buddy! This smart water bottle tracks your intake, reminds you to drink up, and syncs with your favorite health apps. Made from eco-friendly materials, it keeps your drinks cold or hot for hours. Ready to level up your hydration game? Grab yours today and toast to your health! 🥤✨
----------------------------------------



This section introduces a constrained prompt template designed to generate a short and targeted product description. The goal is to guide the language model toward producing output that satisfies explicit structural and stylistic constraints.
- Prompt structuring: The prompt specifies the expected components of the output (headline, features, call to action), as well as stylistic constraints such as tone and word count. This serves to narrow the model’s generative scope and reduce variance.
- Templated input: By using `PromptTemplate`, the content becomes dynamic and reusable. Input values can be easily modified to support different use cases without changing the underlying prompt logic.
- Prompt chaining: The prompt is combined with the language model using the LangChain operator (`|`) to form a processing pipeline. This composition allows the prompt to act as a pre-processor to the LLM.
- Controlled generation: The `.invoke()` method executes the chain, passing in specific input values and retrieving the model’s response. The output adheres to the structure and tone defined in the prompt.

This technique is particularly valuable in scenarios where outputs must follow predefined business rules, maintain a specific voice, or fit within a strict format—such as product copy, summaries, UI text, or catalog entries. By formalizing the output requirements within the prompt itself, this approach offers a simple but powerful way to ensure content quality and consistency in LLM-driven workflows.


## Implementing rule-based generation
In many real-world applications, it is not enough for a language model to simply generate fluent or creative text — the output often needs to follow specific rules or formatting conventions. This is where rule-based generation becomes essential.

In this section, we implement a generation system that produces structured job postings based on clearly defined constraints. By encoding rules directly into the prompt — such as formatting, required sections, and sentence structure — we ensure the output is consistent, predictable, and easy to parse.

This approach is particularly useful when outputs are consumed by other systems or need to adhere to regulatory or editorial standards. It enables greater control over the language model's behavior while preserving its generative capabilities.

The following code demonstrates how to use LangChain’s `PromptTemplate` to design and apply a rule-based prompt that guides the model to generate a formatted job description with clearly delineated sections and standardized language.

In [5]:
# Define a prompt for generating structured job postings based on explicit formatting rules
job_posting_prompt = PromptTemplate(
    input_variables=["job_title", "company", "location", "experience"],
    template="""Create a job posting for a {job_title} position at {company} in {location}.
    The candidate should have {experience} years of experience.
    Follow these rules:
    1. Start with a brief company description (2 sentences)
    2. List 5 key responsibilities, each starting with an action verb
    3. List 5 required qualifications, each in a single sentence
    4. End with a standardized equal opportunity statement

    Format the output as follows:
    COMPANY: [Company Description]

    RESPONSIBILITIES:
    - [Responsibility 1]
    - [Responsibility 2]
    - [Responsibility 3]
    - [Responsibility 4]
    - [Responsibility 5]

    QUALIFICATIONS:
    - [Qualification 1]
    - [Qualification 2]
    - [Qualification 3]
    - [Qualification 4]
    - [Qualification 5]

    EEO: [Equal Opportunity Statement]
    """
)

# Define the specific input values for the generation task
input_variables = {
    "job_title": "Senior Software Engineer",
    "company": "TechInnovate Solutions",
    "location": "San Francisco, CA",
    "experience": "5+"
}

# Chain the prompt with the LLM and invoke it to generate the output
chain = job_posting_prompt | llm
output = chain.invoke(input_variables).content
display_output(output)

Model Output:
----------------------------------------
COMPANY: TechInnovate Solutions is a leading technology firm based in San Francisco, CA, dedicated to delivering cutting-edge software solutions that drive innovation and efficiency for our clients. We foster a collaborative and dynamic work environment that encourages creativity and professional growth.

RESPONSIBILITIES:
- Design and implement scalable software solutions that meet business requirements.
- Collaborate with cross-functional teams to define, design, and ship new features.
- Mentor junior engineers and contribute to their professional development.
- Conduct code reviews to ensure high-quality standards and best practices.
- Troubleshoot and debug applications to optimize performance and reliability.

QUALIFICATIONS:
- A minimum of 5 years of experience in software development with a strong portfolio of completed projects. 
- Proficiency in at least one programming language, such as Java, Python, or JavaScript. 
- Exp

This section demonstrates how to implement rule-based generation by embedding explicit structural and content rules into the prompt. The objective is to guide the language model to output a job posting that adheres to a predefined schema — suitable for automated pipelines or downstream systems requiring structured content.
- Prompt as specification: The prompt doubles as both an instruction and a formatting template. By describing exact rules (e.g. “5 responsibilities”, “1 sentence each”), it constrains the output shape and helps reduce variability and noise in the model's response.
- Structured formatting: The use of consistent headings (`COMPANY:`, `RESPONSIBILITIES:`, etc.) makes the output easier to parse programmatically. This becomes critical when generating content for applications such as automated publishing systems, resume screeners, or HR platforms.
- Input-driven generation: The use of templated variables (`job_title`, `location`, etc.) allows this prompt to be reused flexibly for different job roles or organizations, supporting scalable content generation.
- Prompt chaining and invocation: The prompt is connected to the language model using LangChain's composability operator (`|`), forming a generation chain. The `.invoke()` method is then used to produce the response from the LLM based on the specified inputs.
- Use case alignment: This approach ensures not just grammatical correctness but also conformance to business logic, which is crucial in many enterprise applications where freeform text generation can lead to formatting errors or policy violations.

By clearly encoding both the structure and the output requirements into the prompt, this method ensures reliable, repeatable, and interpretable results from a generative model — effectively bridging the gap between open-ended generation and production-grade structured content.


## Using regex parser for structured output
When working with LLM outputs, especially those intended for downstream processing or integration, it’s important not only to guide the generation format, but also to verify and extract specific parts of the output programmatically.

This is where regular expression (regex) parsing becomes useful. In this section, we demonstrate how to apply a `RegexParser` from LangChain to extract structured components from model output — ensuring that the generated text conforms to a predefined schema.

By combining a structured prompt format with a regex-based parser, we move closer to reliable, machine-readable generations — a crucial step for tasks like report generation, automated documentation, and form filling.

In [6]:
# Define a regex parser to extract structured sections from the model's output
regex_parser = RegexParser(
    regex=r"COMPANY:\s*([\s\S]*?)\n\s*RESPONSIBILITIES:\s*([\s\S]*?)\n\s*QUALIFICATIONS:\s*([\s\S]*?)\n\s*EEO:\s*([\s\S]*)",
    output_keys=["company_description", "responsibilities", "qualifications", "eeo_statement"]
)
# This regex pattern captures the company description, responsibilities, qualifications, and EEO statement from the output text.

# Create a prompt that enforces strict formatting in the model's output
parsed_job_posting_prompt = PromptTemplate(
    input_variables=["job_title", "company", "location", "experience"],
    template="""Create a job posting for a {job_title} position at {company} in {location}.
    The candidate should have {experience} years of experience.
    Follow these rules:
    1. Start with a brief company description (2 sentences)
    2. List 5 key responsibilities, each starting with an action verb
    3. List 5 required qualifications, each in a single sentence
    4. End with a standardized equal opportunity statement

    Format the output EXACTLY as follows:
    COMPANY: [Company Description]

    RESPONSIBILITIES:
    - [Responsibility 1]
    - [Responsibility 2]
    - [Responsibility 3]
    - [Responsibility 4]
    - [Responsibility 5]

    QUALIFICATIONS:
    - [Qualification 1]
    - [Qualification 2]
    - [Qualification 3]
    - [Qualification 4]
    - [Qualification 5]

    EEO: [Equal Opportunity Statement]
    """
)

# Helper function to clean the parsed output (strip whitespace, normalize line breaks)
def clean_output(output):
    for key, value in output.items():
        if isinstance(value, str):
            # Remove leading/trailing whitespace and normalize newlines
            output[key] = re.sub(r'\n\s*', '\n', value.strip())
    return output

# Generate the output from the model using the structured job posting prompt
chain = parsed_job_posting_prompt | llm
raw_output = chain.invoke(input_variables).content

# Parse the model's response using the defined regex pattern
parsed_output = regex_parser.parse(raw_output)
# Clean and normalize the extracted content
cleaned_output = clean_output(parsed_output)

# Display each section in a readable format
print("Parsed Output:")
for key, value in cleaned_output.items():
    print(f"{key.upper()}:")
    print(value)
    print()

Parsed Output:
COMPANY_DESCRIPTION:
TechInnovate Solutions is a leading technology firm based in San Francisco, CA, specializing in cutting-edge software development and innovative technology solutions. Our mission is to empower businesses through advanced technology that drives efficiency and growth.

RESPONSIBILITIES:
- Design and implement scalable software solutions to enhance user experience.
- Collaborate with cross-functional teams to define project requirements and specifications.
- Mentor junior engineers and foster a culture of continuous improvement and learning.
- Conduct code reviews to ensure quality and adherence to best practices.
- Troubleshoot and resolve complex technical issues in a timely manner.

QUALIFICATIONS:
- A minimum of 5 years of professional software engineering experience is required.
- Proficiency in programming languages such as Java, Python, or JavaScript is essential.
- Experience with cloud platforms such as AWS, Azure, or Google Cloud is preferred.

1. Regex definition: A `RegexParser` is created with a custom pattern to extract four expected sections from the generated text. The pattern looks for labeled blocks — `COMPANY`, `RESPONSIBILITIES`, `QUALIFICATIONS`, and `EEO` — and captures everything under each header using non-greedy matches.
2. Prompt enforcement: The `PromptTemplate` enforces exact formatting instructions for the language model. This ensures the output will contain the sections in a predictable order, which makes the regex parsing reliable.
3. Text generation: The model is invoked with the formatted prompt and job-related input variables. The generated output is expected to follow the given format exactly.
4. Parsing and cleaning: The raw model output is parsed using the regex, extracting each section into a dictionary. The `clean_output` function is then used to trim whitespace and normalize line breaks for cleaner presentation and easier downstream processing.
5. Output presentation: Finally, each extracted section is printed in uppercase, mimicking structured display formatting that could be further exported or logged.

This method demonstrates a powerful pattern in constrained generation: combining prompt-level control with post-generation validation and parsing. It enhances the reliability of LLMs in structured content generation tasks, especially when the outputs must be consumed by other systems or meet compliance standards.

## Implementing additional constraints
This section demonstrates how to define multi-layered constraints in a text generation task. By embedding multiple detailed requirements into the prompt, it is possible to guide the language model toward producing tightly structured outputs with specific form, style, and content limits.

This technique is particularly useful for tasks like review generation, summarization, or template-based content creation, where both layout and semantics must follow well-defined rules. By explicitly enumerating expectations — such as content count, sentence length, tone, or word count — the prompt becomes a robust instruction set for the model to follow.

In [7]:
# Create a prompt template with detailed constraints for a structured product review
review_prompt = PromptTemplate(
    input_variables=["product", "rating", "pros", "cons", "word_limit"],
    template="""Write a product review for {product} with the following constraints:
    1. The review should have a {rating}-star rating (out of 5)
    2. Include exactly {pros} pros and {cons} cons
    3. Use between 2 and 3 sentences for each pro and con
    4. The entire review should be under {word_limit} words
    5. End with a one-sentence recommendation

    Format the review as follows:
    Rating: [X] out of 5 stars

    Pros:
    1. [Pro 1]
    2. [Pro 2]
    ...

    Cons:
    1. [Con 1]
    2. [Con 2]
    ...

    Recommendation: [One-sentence recommendation]
    """
)

# Define specific inputs for the review
input_variables = {
    "product": "Smartphone X",      # Product name
    "rating": "4",                  # Star rating
    "pros": "3",                    # Number of pros to include
    "cons": "2",                    # Number of cons to include
    "word_limit": "200"            # Maximum allowed word count
}

# Build the chain: prompt -> LLM
chain = review_prompt | llm
# Invoke the chain and get the model's output
output = chain.invoke(input_variables).content
# Display the generated review
display_output(output)

Model Output:
----------------------------------------
Rating: 4 out of 5 stars

Pros:
1. The camera quality on Smartphone X is impressive, capturing vibrant and detailed images even in low light. The advanced features like night mode and optical zoom elevate photography to a professional level.
2. Battery life is outstanding, lasting a full day with heavy usage. Quick charging capabilities also mean you can get back to using your phone without long waits.
3. The sleek design and high-quality display make using the Smartphone X a pleasure. The vibrant colors and sharp resolution enhance everything from gaming to streaming videos.

Cons:
1. The price point is a bit high compared to similar models on the market, which may deter budget-conscious buyers. While it offers great features, some might find better value elsewhere.
2. The lack of expandable storage can be limiting for users who store a lot of media. This could lead to potential frustrations as the device fills up quickly.

Recomm

1. Prompt construction with multi-step constraints: The prompt includes detailed rules governing the structure, quantity, and length of the generated review content. The constraints apply not only to the sections (e.g., number of pros and cons), but also to the sentence count per item and total word count. A clear format is enforced so the model knows exactly how to arrange its output.
2. Input customization: The dictionary `input_variables` defines the parameters for a particular product review. This makes the template reusable — the same prompt logic can be applied to any product by simply changing the inputs.
3. Chain execution: The prompt is connected to the language model via a LangChain `Runnable` (represented by `| llm`), and the final output is generated using `.invoke()` with the defined parameters.

This example illustrates how to simulate programmatic control over free-form generation by using prompt engineering alone — no post-processing or external validators are needed to enforce rules. With clear formatting and quantified constraints, models like GPT can reliably produce content that fits highly structured specifications.

This technique can be extended to more complex domains, such as report generation, automated assessments, or personalized recommendations — any case where the output must satisfy a format contract or domain-specific checklist.